### Graham Kerr
#### graham.s.kerr@NASA.gov; kerrg@cua.edu

<H1 font size="+3" style="color:red">
Turbulence in the Active Sun<br>
-- Flare Overview <br>
</H1>

<b>This notebook will</b>
> - Make a figure showing the SJI image, with SG alongside (Si IV, O IV, Fe XXI)

---
### <b style="color:blue"> Some set up </b>

***Import Modules***

> - Note that here I have called a module called 'utils' that contains a few functions that I have collected. The python session knows where they are held as I insert their location to the path. You will have to modify that line to point to where you put utils.

In [ ]:
##
## Import various modules -- can tidy up later
##
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, PowerNorm
#%matplotlib tk
import sys
sys.path.insert(0,'/Users/gskerr1/Documents/Research/Python_Programs/MiscCodes/')
sys.path.insert(0,'/Users/gskerr1/Documents/Research/Python_Programs/')
import utils
import os
import re
from scipy import io
import pickle
import cmocean
import copy
import iris_lmsalpy
# import astropy
import pandas as pd
# import hissw
# from astropy.coordinates import SkyCoord
# import astropy.units as u
# from sunpy.coordinates import frames
# import sunpy.map

---
### <b style="color:blue"> Set up some plotting parameters </b>
> - These are just my personal preferences for plot styles... you dont need to use them, or can change them my modifying the params in the function, or generally changing rcParams (check the matplotlib docs)

In [ ]:
plot_dict = utils.plotsetup_image(font_size = 22)
plt.rcParams.update(plot_dict)

---
### <b style="color:blue"> Read in the data </b>


***SG***

In [ ]:
#data read in - will take a moment
dir1 = '/Volumes/GrahamKerr_work/OI_IRIS_data/2014_10_25_1500/'
file_sg = dir1+r'IRIS_SG_Oct_25_siiv1394_expcorr.pkl'
with open(file_sg, 'rb') as output:  
    sg_dict = pickle.load(output)
dir1 = '/Volumes/GrahamKerr_work/OI_IRIS_data/2014_10_25_1500/'
file_sg = dir1+r'IRIS_SG_2014_Oct_25_oi_expcorr.pkl'
with open(file_sg, 'rb') as output:  
    sg_dict_oi = pickle.load(output)

In [ ]:
##
## Load the SJI data  
##
dir1 = '/Volumes/GrahamKerr_work/OI_IRIS_data/2014_10_25_1500/'
file_sji = dir1+'IRIS_SJI_2014_Oct_25_exptimecorr.pkl'
with open(file_sji, 'rb') as output:  
    sji_dict = pickle.load(output)

In [ ]:
savepng = True
savepdf = False

dirout = '/Users/gskerr1/Documents/Research/Turbulence_HSR/'
fname_out = 'IRISSJI_1330A_2014_Oct_25_wSpectra'


inds = []
for ii in range(len(sji_dict['time_1330'])):
    if sji_dict['time_1330'][ii][0:10] != '2014-10-25': inds.append(ii)
timer_tmp = copy.deepcopy(sji_dict['time_1330'])
timer = pd.to_datetime(np.delete(timer_tmp, inds))
data_tmp = copy.deepcopy(sji_dict['sji_1330'])
data_sji = np.delete(sji_dict['sji_1330'], inds, axis=-1)


data = copy.deepcopy((data_sji))
index = sji_dict['hdr_1330_ext0']

times2plot = np.arange(1200, 1800, 1)
# times2plot = [1551]

cmap1330 = cmocean.cm.amp_r
cmapspectra = cmocean.cm.haline

xsize = 15
ysize = 8

dx_sji = 0.5
dy_sji = 0.8
y1 = 0.05
x1 = 0.05

dx_sg = 0.2
dy_sg = 0.8
sx_sg = 0.05
sx_sji = -0.015

xcoord1 = 0
xcoord2 = data.shape[0]
# xcoord2 = 260
ycoord1 = 0
# ycoord2 = 300
ycoord2 = data.shape[1]



counter = 0

wl1_sg1 = 1353.0
wl2_sg1 = 1356.5

wl1_sg2 = 1393.0
wl2_sg2 = 1395.0

wl1_sg3 = 1400.5
wl2_sg3 = 1401.75

for ii in range(0, len(times2plot)):
    
    tind_sg = times2plot[ii]
    tind_sji = np.abs(timer - pd.to_datetime(sg_dict['hdr']['dateobs'][tind_sg])[0]).argmin() 
    
    fig = plt.figure(figsize=(xsize, ysize))
    ax1 = fig.add_axes([x1, y1, dx_sji, dy_sji])
    ax2 = fig.add_axes([x1+1*dx_sji+1*sx_sji, y1, dx_sg, dy_sg])
    ax3 = fig.add_axes([x1+1*dx_sji+1*sx_sji+1*dx_sg+1*sx_sg, y1, dx_sg, dy_sg])
    ax4 = fig.add_axes([x1+1*dx_sji+1*sx_sji+2*dx_sg+2*sx_sg, y1, dx_sg, dy_sg])

    ax1.set_ylabel('Solar - Y [pixels]')
    ax1.set_xlabel('Solar - X [pixels]')

    ax2.set_ylabel('Solar - X [pixels]')
    ax2.set_xlabel('Wavelength $[\mathrm{\AA}$]')
    ax3.set_xlabel('Wavelength $[\mathrm{\AA}$]')
    ax4.set_xlabel('Wavelength $[\mathrm{\AA}$]')
    ax5.set_xlabel('Wavelength $[\mathrm{\AA}$]')

    
    cbartitle = 'log$_{10}$ I [DN px$^{-1}$ s$^{-1}$]'
    
    ax1.xaxis.set_tick_params(rotation=20)
    ax1.tick_params(which='both', direction='inout')
    ax2.xaxis.set_tick_params(rotation=20)
    ax2.tick_params(which='both', direction='inout')
    ax3.xaxis.set_tick_params(rotation=20)
    ax3.tick_params(which='both', direction='inout')
    ax4.xaxis.set_tick_params(rotation=20)
    ax4.tick_params(which='both', direction='inout')

    ax3.tick_params(
        axis='both',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=True,         # ticks along the top edge are off
        labelbottom=True,
        labeltop = False,
        left=True,      # ticks along the bottom edge are off
        right=True,         # ticks along the top edge are off
        labelright=False,
        labelleft = False
        ) 
    ax4.tick_params(
        axis='both',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=True,         # ticks along the top edge are off
        labelbottom=True,
        labeltop = False,
        left=True,      # ticks along the bottom edge are off
        right=True,         # ticks along the top edge are off
        labelright=False,
        labelleft = False
        ) 

    #### SJI PANEL
    norm_sji = LogNorm(vmin = 25, vmax = 3000, clip = True)
    im = ax1.imshow(np.rot90(data[:,:,tind_sji]),
                    cmap=cmap1330, 
                    origin='lower',interpolation='nearest', aspect='equal',
                    extent = [xcoord1, xcoord2, ycoord1, ycoord2],
                    norm = norm_sji)


    #### SG PANEL 1
    wind1_sg1 = np.abs(sg_dict_oi['wl'][:,0] - wl1_sg1).argmin()
    wind2_sg1 = np.abs(sg_dict_oi['wl'][:,0] - wl2_sg1).argmin()+1
    norm_sg1 = PowerNorm(0.25, vmin=0, vmax=np.nanmax(sg_dict_oi['data'][:,1200:1800,wind1_sg1:wind2_sg1,0])/4)
    im = ax2.imshow((sg_dict_oi['data'][:,tind_sg,wind1_sg1:wind2_sg1,0]),
                    cmap=cmapspectra, 
                    origin='lower',interpolation='nearest', aspect='auto',
                    extent = [wl1_sg1, wl2_sg1,
                              xcoord1, xcoord2],
                    norm = norm_sg1)


    #### SG PANEL 2
    wind1_sg2 = np.abs(sg_dict['wl'][:,0] - wl1_sg2).argmin()
    wind2_sg2 = np.abs(sg_dict['wl'][:,0] - wl2_sg2).argmin()+1
    norm_sg2 = PowerNorm(0.25, vmin=0, vmax=np.nanmax(sg_dict['data'][:,1200:1800,wind1_sg2:wind2_sg2,0])/4)
    im = ax3.imshow((sg_dict['data'][:,tind_sg,wind1_sg2:wind2_sg2,0]),
                    cmap=cmapspectra, 
                    origin='lower',interpolation='nearest', aspect='auto',
                    extent = [wl1_sg2, wl2_sg2,
                              xcoord1, xcoord2],
                    norm = norm_sg2)

    #### SG PANEL 3
    wind1_sg3 = np.abs(sg_dict['wl'][:,0] - wl1_sg3).argmin()
    wind2_sg3 = np.abs(sg_dict['wl'][:,0] - wl2_sg3).argmin()+1
    norm_sg3 = PowerNorm(0.25, vmin=0, vmax=np.nanmax(sg_dict['data'][:,1200:1800,wind1_sg3:wind2_sg3,0])/4)
    im = ax4.imshow((sg_dict['data'][:,tind_sg,wind1_sg3:wind2_sg3,0]),
                    cmap=cmapspectra, 
                    origin='lower',interpolation='nearest', aspect='auto',
                    extent = [wl1_sg3, wl2_sg3,
                              xcoord1, xcoord2],
                    norm = norm_sg3)





    ax1.set_title('IRIS/SJI 1330A\n'+sji_dict['time_1330'][tind_sji]+'UT', fontsize = 18)
    ax2.set_title('IRIS/SG FeXXI, OI, CI\n'+sg_dict['hdr']['dateobs'][tind_sg][0]+'UT', fontsize = 18)
    ax3.set_title('IRIS/SJI SiIV\n'+sg_dict['hdr']['dateobs'][tind_sg][0]+'UT', fontsize = 18)
    ax4.set_title('IRIS/SJI OVI\n'+sg_dict['hdr']['dateobs'][tind_sg][0]+'UT', fontsize = 18)

    ax2.text(0.05, 0.85, 'frame #%d' %(tind_sg), color = 'white', transform = ax2.transAxes, fontsize = 18)
    ax3.text(0.05, 0.85, 'frame #%d' %(tind_sg), color = 'white', transform = ax3.transAxes, fontsize = 18)
    ax4.text(0.05, 0.85, 'frame #%d' %(tind_sg), color = 'white', transform = ax4.transAxes, fontsize = 18)


    
#     ax1.text(1.05, 0.95, 'IRIS/SJI 2832A', color = 'black', transform = ax1.transAxes, fontsize = 15)
#     ax1.text(1.05, 0.90, sji_dict['time_2832'][tind]+'UT'  , color = 'black',transform=ax1.transAxes,fontsize=15)     
    # ax1.axhline(y=sji_dict['SLTPX1IX_1330'][tind_sji]-3, color = 'white', linewidth=0.5, linestyle='--', zorder = 20)

    # cbar_ax1 = fig.add_axes([x1+1*dx+0*sx+0.025, y1+0*dy+0*sy, 0.025, dy])
    # norm = mpl.colors.Normalize(vmax=0.5, vmin=3.5)
    # cb1 = mpl.colorbar.ColorbarBase(cbar_ax1, 
    #                                 cmap=cmap1330, 
    #                                 norm=norm,
    #                                 orientation = 'vertical') 
    # cbar_ax1.tick_params(labelsize=17, width=2.0)
    # cbar_ax1.minorticks_off()
    # cb1.set_label(cbartitle)

    r2_x1 = 40
    r2_x2 = 80
    r1_x1 = 180
    r1_x2 = 250
    
    rect1 = mpl.patches.Rectangle((r2_x1,150),
                                   (r2_x2-r2_x1), 60,
                                   ec ='dodgerblue', 
                                   fc = 'none',
                                   lw = 3, 
                                   ls = '--')
    rect2 = mpl.patches.Rectangle((r1_x1,150),
                                   (r1_x2-r1_x1), 60,
                                   ec ='dodgerblue', 
                                   fc = 'none',
                                   lw = 3, 
                                   ls = '--')
  
    ax1.add_patch(rect1)
    ax1.add_patch(rect2)
    ax1.text(0.05, 0.6, 'Region 2', color = 'dodgerblue', transform = ax1.transAxes, fontsize = 18)
    ax1.text(0.7, 0.6, 'Region 1', color = 'dodgerblue', transform = ax1.transAxes, fontsize = 18)

    rect3 = mpl.patches.Rectangle((wl1_sg1+0.05,r2_x1),
                                   (wl2_sg1-wl1_sg1-0.1), (r2_x2-r2_x1),
                                   ec ='tomato', 
                                   fc = 'none',
                                   lw = 3, 
                                   ls = '--')
    ax2.add_patch(rect3)
    rect4 = mpl.patches.Rectangle((wl1_sg2+0.05,r2_x1),
                                   (wl2_sg2-wl1_sg2-0.1), (r2_x2-r2_x1),
                                   ec ='tomato', 
                                   fc = 'none',
                                   lw = 3, 
                                   ls = '--')
    ax3.add_patch(rect4)
    rect5 = mpl.patches.Rectangle((wl1_sg3+0.05,r2_x1),
                                   (wl2_sg3-wl1_sg3-0.1), (r2_x2-r2_x1),
                                   ec ='tomato', 
                                   fc = 'none',
                                   lw = 3, 
                                   ls = '--')
    ax4.add_patch(rect5)



    rect6 = mpl.patches.Rectangle((wl1_sg1+0.05,r1_x1),
                                   (wl2_sg1-wl1_sg1-0.1), (r1_x2-r1_x1),
                                   ec ='tomato', 
                                   fc = 'none',
                                   lw = 3, 
                                   ls = '--')
    ax2.add_patch(rect6)
    rect7 = mpl.patches.Rectangle((wl1_sg2+0.05,r1_x1),
                                   (wl2_sg2-wl1_sg2-0.1), (r1_x2-r1_x1),
                                   ec ='tomato', 
                                   fc = 'none',
                                   lw = 3, 
                                   ls = '--')
    ax3.add_patch(rect7)
    rect8 = mpl.patches.Rectangle((wl1_sg3+0.05,r1_x1),
                                   (wl2_sg3-wl1_sg3-0.1), (r1_x2-r1_x1),
                                   ec ='tomato', 
                                   fc = 'none',
                                   lw = 3, 
                                   ls = '--')
    ax4.add_patch(rect8)

    ax2.plot([1355, 1355, 1355], [r2_x1+7, r2_x1+8, r2_x1+10], marker = 'x', color = 'tomato', linewidth = 0) 
    ax3.plot([1394.5, 1394.5, 1394.5], [r2_x1+7, r2_x1+8, r2_x1+10], marker = 'x', color = 'tomato',linewidth = 0)
    ax4.plot([1400.75, 1400.75, 1400.75], [r2_x1+7, r2_x1+8, r2_x1+10], marker = 'x', color = 'tomato',linewidth = 0) 

    ax2.plot([1355, 1355, 1355], [r1_x1+14, r1_x1+15, r1_x1+23], marker = 'x', color = 'tomato',linewidth = 0) 
    ax3.plot([1394.5, 1394.5, 1394.5], [r1_x1+14, r1_x1+15, r1_x1+23], marker = 'x', color = 'tomato',linewidth = 0)
    ax4.plot([1400.75, 1400.75, 1400.75], [r1_x1+14, r1_x1+15, r1_x1+23], marker = 'x', color = 'tomato',linewidth = 0)

    ax1.plot([r1_x1+14, r1_x1+15, r1_x1+23], [181,181,181], marker = 'x', color = 'dodgerblue',linewidth = 0) 
    ax1.plot([r2_x1+7, r2_x1+8, r2_x1+10], [181,181,181], marker = 'x', color = 'dodgerblue',linewidth = 0) 

    ax1.set_xlim(30, 260)
    ax2.set_ylim(30, 260)
    ax3.set_ylim(30, 260)
    ax4.set_ylim(30, 260)

    if savepng == True:
        plt.savefig(dirout+'/png/'+fname_out+'_%04d.png' %(ii), format='png', bbox_inches = 'tight', dpi=300)
    if savepdf == True:
        plt.savefig(dirout+'/pdf/'+fname_out+'_%04d.pdf' %(ii), format='pdf', bbox_inches = 'tight', dpi=300)
    
    
    
    plt.show()

In [ ]:
dirout+